## Pertemuan 10

- Edge Detection
- Contour Detection

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

___
## 1. Edge Detection (Canny Edge Detection)

- Canny Edge detection merupakan algoritma edge detection paling populer. 
- Dapat digunakan untuk menjari batas-batas objek pada gambar.
- menggunakan method `cv2.canny(img, threshMin, threshMax)`
- Canny Edge detection diilustrasikan sebagai berikut,

![](resource/canny.png)

- Mendapatkan **Edge image** dari gambar RGB (cara 1)
    - Load Image
    - Convert to Gray
    - Apply Binary Thresholding
    - Find Edge Image
    - Show Image

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply binary tresholding
ret, thresh = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY_INV)

#cv2.canny(<image>, threshold_min, threshold_max)
edged = cv2.Canny(thresh, 200, 210)

#show image 
plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(img[:,:,::-1]) # 3 channel image

plt.subplot(2, 2, 2)
plt.title("Gray Image")
plt.imshow(gray, cmap="gray") # 3 channel image

plt.subplot(2, 2, 3)
plt.title("Binary Image")
plt.imshow(thresh, cmap="gray") # 1 channel image

plt.subplot(2, 2, 4)
plt.title("Edge Image")
plt.imshow(edged, cmap="gray") # 1 channel image


- Mendapatkan **Edge Image** dari Image RGB (cara 2)
    - Load Image
    - Convert to HSV
    - Apply Range Thresholding
    - Find Edge Image
    - Show Image

In [ ]:
# define range of red color in HSV
lower_red = np.array([-10, 50, 50])
upper_red = np.array([10, 255, 255])

# define range of green color in HSV
lower_green = np.array([35, 50, 50])
upper_green = np.array([70, 255, 255])

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)

#cv2.canny(<image>, threshold_min, threshold_max)
edged = cv2.Canny(res, 200, 210)

#show image 
plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(img[:,:,::-1]) # 3 channel image

plt.subplot(2, 2, 2)
plt.title("Mask Image")
plt.imshow(mask, cmap="gray") # 1 channel image

plt.subplot(2, 2, 3)
plt.title("Segmented Image")
plt.imshow(res[:,:,::-1]) # 3 channel image

plt.subplot(2, 2, 4)
plt.title("Edge Image")
plt.imshow(edged, cmap="gray") # 1 channel image


## Task
- Lengkapi Class Preprocessing agar dapat menerapkan Range Thresholding dan melakukan Edge Detection 
    - Menggunakan Range Tresholding

In [ ]:
class Preprocessing : 
    def __init__(self, DATASET_FOLDER = "Dataset_Tomat/"):
        self.labels = []
        self.image_list = []
        self.image_range = []
        self.image_edged = []
        self.DATASET_FOLDER = DATASET_FOLDER
        
        # define range of red color in HSV
        self.lower_red = np.array([-10, 50, 50])
        self.upper_red = np.array([10, 255, 255])

        # define range of green color in HSV
        self.lower_green = np.array([35, 50, 50])
        self.upper_green = np.array([70, 255, 255])
        
    def ImageRead(self):
        for folder in os.listdir(self.DATASET_FOLDER):
            for file in os.listdir(self.DATASET_FOLDER + folder):
                img = cv2.imread(self.DATASET_FOLDER + folder + "/" + file)
                self.image_list.append(img)
                self.labels.append(folder) # append label (name) of image
                            
    def RangeTresholding(self):
        # loop  for all self.image_list           
            # convert to hsv
            # .......................

            # apply range tresholding
            # .......................
            # .......................
            # .......................
            # append to self.image_range
            
    def EdgeDetection(self):
        # loop  for all self.image_range
            # apply edge detection
            # .......................
            # .......................
            # append to self.imaged_edged
            
    def SaveAllEdged(self, EDGED_FOLDER = "edged_tomato/"):
        if not os.path.exists(EDGED_FOLDER) :
            os.mkdir(EDGED_FOLDER)
            
        for i in range(len(self.image_edged)):

            # get image
            img = self.image_edged[i]

            # check if folder exist. if not, create that folder    
            folder_path = EDGED_FOLDER + self.labels[i] + "/"
            if not os.path.exists(folder_path) :
                os.mkdir(folder_path)

            # save image
            file_name = self.labels[i] + "_%03d.jpg" % i
            file_path = EDGED_FOLDER + self.labels[i] + "/" + file_name

            cv2.imwrite(file_path, img)

In [ ]:
preprocess1 = Preprocessing(DATASET_FOLDER = "Dataset_Tomat/")

preprocess1.ImageRead()


In [ ]:
preprocess1.RangeTresholding()
preprocess1.EdgeDetection()

In [ ]:
preprocess1.SaveAllEdged()

## 2. Contour Detection & Contour Drawing 

- Menggunakan method `cv2.findContour(img, mode, method)`
- dimana :
    - `img` : input image
    - `mode` :
        - `cv2.CHAIN_APPROX_NONE` : all the boundary points are stored
        - `cv2.CHAIN_APPROX_SIMPLE` : only end points / corner of that shape are stored <br>
        <img src="resource/mode_find_contour.png" style="width:300px; margin-top:10px"></img>
        - `with cv2.CHAIN_APPROX_NONE` (734 points) 
        - second image shows the one with `cv2.CHAIN_APPROX_SIMPLE` (only 4 points)
    - `method` : 
        - `cv2.RETR_EXTERNAL`  : retrieves only the **extreme outer contours**.  It sets all contours to `hierarchy[i][2] = hierarchy[i][3] = -1`.
        - `cv2.RETR_LIST`   : retrieves **all of the contours** without establishing any hierarchical relationships. 
        - `cv2.RETR_CCOMP`  : retrieves **all of the contours** and organizes them into a **two-level hierarchy**.  
        - `cv2.RETR_TREE`   : retrieves **all of the contours** and reconstructs a **full hierarchy** of nested contours. 
        
- Output : 
    - `contours` : list of countour location (x,y) : <br>
    <img src="resource/hierarchy_moves.gif" style="width:400px; margin-top:10px"></img>
    
    - `hierarchy` : list of `[Next, Previous, First_Child, Parent]`, <br> Representation of this relationship when some shapes are inside other shapes, we call outer one as **parent** and inner one as **child**. 
        - `Next` : next contour at the same hierarchical level.
        - `Previous` : previous contour at the same hierarchical level.
        - `First_Child` : first child contour.
        - `Parent` : index of its parent contour. <br>
        > *If there is no **child** or **parent**, that field is taken as -1*

### 2.1 Find Contour from Binary Thresholding Image `cv2.RETR_EXTERNAL`
- retrieves only the extreme outer contours. It sets all contours to hierarchy[i][2] = hierarchy[i][3] = -1.

In [ ]:
img = cv2.imread('hierarchy.png')

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# convert to binary image
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# find contour
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [ ]:
contours

 - **Draw Contour `cv2.drawContour()`**

- menggunakan method `cv2.drawContours(img, contour, contour_index, (B,G,R), thickness)`
- dimana :
    - `img` : input image
    - `contour` : contour location (list)
    - `contour_index` : parameter indicating a contour to draw. If it is negative, all the contours are drawn.
    - `(B,G,R)` : contour color
    - `thickness` : contour thickness

In [ ]:
Draw contour to image
for cnt in contours:
    cv2.drawContours(img, cnt, -1, (255,0,0), 3)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img[:,:,::-1])

- Find Contour & Draw Contour for `Tomat.jpg`

In [ ]:
img = cv2.imread('Tomat.jpg')

# convert to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


# convert to binary image
ret, thresh = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY_INV)

# find contour
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to original image
for cnt in contours:
    cv2.drawContours(img, cnt, -1, (255,0,0), 2)
    
plt.figure(figsize=(7,7))
plt.imshow(img[:,:,::-1])

### 2.2 Find Contour from Range Tresholding Image & Edge Image

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)


# apply edge detection
edged = cv2.Canny(res, 200, 210)

# find contour
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# draw contour to original image
for cnt in contours:
    cv2.drawContours(img, cnt, -1, (255,0,0), 2)
    
plt.figure(figsize=(7,7))
plt.imshow(img[:,:,::-1])

### Task
- Modifikasi class Preprocessing diatas, tambahkan method **FindContours()** yang akan mencari contour image dan draw contour dari list **self.image_edged** (image hasil range tresholding + edge detection)
- Tambahkan method **SaveAllContourImage()** untuk save image yang dusah di draw contour

____
## 2.3 Contour Feature 
- Contour Area (luasan) <br>
    `area = cv2.contourArea(cnt)`
- Bounding Rectangle <br>
    <img src="resource/boundingrect.png" style="width:200px; margin-top:10px;"></img>
    - Straight Bounding Rectangle <br>
        `rect = cv2.boundingRect(cc)`
    - Rotated Rectangle <br>
        `rect = cv2.minAreaRect(cnt)`

- Contour Area (luasan)

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)


# apply edge detection
edged = cv2.Canny(res, 200, 210)

# find contour
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [ ]:
for cnt in contours:
    area = cv2.contourArea(cnt)
    print("luas : %d pixel" % area)

- Straight Bounding Rectangle

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)


# apply edge detection
edged = cv2.Canny(res, 200, 210)

# find contour
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# find bounding rect
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    print(rect) # x, y, w, h

- Draw bounding box for each contour using `cv2.rectangle()` \
`cv2.rectangle(image_src, (x0,y0), (xt,yt), (B,G,R), thickness)`

In [ ]:
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    x, y, w, h = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
    
plt.figure(figsize=(7,7))
plt.imshow(img[:,:,::-1])

### 2.4 Filter **small** contour area

In [ ]:
# funtion for small contour filtering
def filter_contour(contours):
    filtered_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        if not (area < 100 or w < 50 or h < 50) :
            filtered_contours.append(cnt)
    return filtered_contours

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)


# apply edge detection
edged = cv2.Canny(res, 200, 210)

# find contour
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# filter contour
contours = filter_contour(contours)

# find bounding rect
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    print(rect) # x, y, w, h
    cv2.rectangle(img, (x, y), (x+w, y+h), (0,0,255), 3)
    
plt.figure(figsize=(7,7))
plt.imshow(img[:,:,::-1])

### 2.5 Crop each contour from input image, and display

In [ ]:
# load image
img = cv2.imread('Tomat.jpg')

#convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Threshold the HSV image to get only red & green colors
mask_green = cv2.inRange(hsv.copy(), lower_green, upper_green)
mask_red = cv2.inRange(hsv.copy(), lower_red, upper_red)
mask = mask_green + mask_red
res = cv2.bitwise_and(img, img, mask= mask)


# apply edge detection
edged = cv2.Canny(res, 200, 210)

# find contour
contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# filter contour
contours = filter_contour(contours)

# find bounding rect
for cnt in contours:
    rect = cv2.boundingRect(cnt)
    print(rect) # x, y, w, h
    roi = img[y:y+h, x:x+w]
    
    
plt.figure(figsize=(7,7))
plt.imshow(roi[:,:,::-1])

### Home Work
- Tambahkan method **CropByContours()** kedalam class Preprocessing seperti yang diimplementasikan diatas
- Tambahkan method **filter_contour()** kedalam class Preprocessing seperti yang diimplementasikan diatas
- Save Hasil croping by contour ke folder **Crop_Image/**